In [ ]:
import random
def random_neighbor_selection(S, S_prime):
  #Takes sorted S input; sort before call
  n = len(S) 
  T = S_prime.copy()
  idxs = random.sample(range(0, n), 2)
  idx1 = idxs[0]
  idx2 = idxs[1]
  #print(idx1, idx2)
  elem1 = S[idx1]
  if elem1 in T:
    T.remove(elem1)
  else:
    T.add(elem1)
  
  if random.uniform(0, 1) > 0.5:
    elem2 = S[idx2]
    if elem2 in T:
      T.remove(elem2)
    else:
      T.add(elem2)

  if len(T) == 0:
    T = S_prime.copy()

  return T

In [ ]:
def initial_random_subset(S):
  current_subset = random.sample(S, 2)
  return set(current_subset)

In [ ]:
def residue(S_prime, W):
  res = None
  sum_S_prime = sum(S_prime)
  if sum_S_prime > W:
    res = sum_S_prime
  else:
    res = W - sum_S_prime
    assert (res >= 0)
  return res

In [ ]:
import math
def consider_bad_neighbor(i, current, neighbor, temp_factor=0.8):
  #This formula is for sets of size around 100. 
  X = (neighbor - current)/(10000000000 * (temp_factor**(i/300)))
  P = math.exp(-X)
  if random.uniform(0, 1) < P:
    return True
  else:
    return False

In [ ]:
def simulated_annealing_ssp(S, W, r=10000, temp_factor=0.8):
  current_subset = initial_random_subset(S)
  current_residue = residue(current_subset, W)
  min_residue = 999999
  for i in range(r):
    iter = i + 1
    random_neighbor = random_neighbor_selection(sorted(S), current_subset)
    neighbor_residue = residue(random_neighbor, W)
    if neighbor_residue < current_residue:
      current_subset = random_neighbor
      current_residue = neighbor_residue
    else:
      if consider_bad_neighbor(iter, current_residue, neighbor_residue, temp_factor):
        current_subset = random_neighbor
        current_residue = neighbor_residue
    if current_residue < min_residue:
      min_residue = current_residue
    if current_residue == 0:
      print("Target found!")
      break
  return W - min_residue

In [ ]:
def print_(msg, out_path):
  print(msg)
  with open(out_path, "a") as f:
    f.write(msg + "\n")

In [ ]:
import numpy as np
import time
import os
#id = 1000
id = input("Input 10 or 100 or 1000: ")
input_file = "./test_" + str(id) + ".txt"
soln_file = "./test_" + str(id) + "_soln.txt"
out_path = "./output_" + str(id) + ".txt"

if os.path.exists(out_path):
  os.remove(out_path)

with open(input_file, "r") as f:
  lines = f.readlines()
inputs = []
for line in lines:
  tmp = line.strip()
  if len(tmp)>0:
    tmp = int(tmp)
    inputs.append(tmp)

read_upto = 0
T = inputs[read_upto]
read_upto += 1

sa_soln = []
times = []
for t in range(T):
  N = inputs[read_upto]
  read_upto += 1
  weights = []
  for i in range(N):
    weights.append(inputs[read_upto])
    read_upto += 1
  weights = np.array(weights)
  target = inputs[read_upto]
  read_upto += 1

  start = time.time()
  val = simulated_annealing_ssp(set(weights), target, 100000)
  end = time.time()
  sa_soln.append(val)
  times.append(end-start)

sa_soln = np.array(sa_soln)

with open(soln_file, "r") as f:
  lines = f.readlines()
opt_soln = [int(line.strip()) for line in lines]
opt_soln = np.array(opt_soln)

exact_cnt = 0
for g, o, t in zip(sa_soln, opt_soln, times):
  #print(int(g), o)
  msg = str(g) +" " + str(o) + " " + str(t) + "seconds"
  print_(msg, out_path) 
  if(int(g)==o):
    exact_cnt += 1

print(exact_cnt)
avg_apx = (1/10)*np.sum(np.divide(sa_soln,opt_soln))
print_("Average approximation: " + str(avg_apx), out_path)
avg_time = sum(times)/len(times)
print_("Average time: " + str(avg_time) + "seconds", out_path)

Input 10 or 100 or 1000: 1000
Target found!
Target found!
Target found!
Target found!
Target found!
Target found!
Target found!
Target found!
Target found!
Target found!
248299 248299 1.8949270248413086seconds
250617 250617 1.8690907955169678seconds
253109 253109 2.2035651206970215seconds
240667 240667 1.9300580024719238seconds
238479 238479 1.8236346244812012seconds
247621 247621 1.8725321292877197seconds
257666 257666 1.8383395671844482seconds
239188 239188 2.0270144939422607seconds
241480 241480 1.8182916641235352seconds
240590 240590 2.101280927658081seconds
10
Average approximation: 1.0
Average time: 1.9378734350204467seconds
